In [15]:
%tensorflow_version 1.x
SIZE = "2k4"
C = "b"
CLASSES = 2
FOLDER = ""
FOLDS_PATH = "".format(SIZE,C)
DATA_FOLDER = ""
BATCH_SIZE = 32

CLASS_NAME = {2:['Sem Risco', 'Risco Potencial'], 3:['Sem Risco', 'Risco Potencial', 'Risco Alto']}

In [6]:
import numpy as np 
import pandas as pd 
import os
from pathlib import Path
from fastai.text import *
from time import ctime


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Loading dataset and doing cross validation

In [16]:
folds = int(len([name for name in os.listdir(FOLDS_PATH) if os.path.isfile(os.path.join(FOLDS_PATH,name))]) / 2)
print("Training on {} folds".format(folds))

val_acc = []
val_prec = []
val_rec = []
val_f1 = []


for f in range(10):  
  df_train = pd.read_csv(FOLDS_PATH + "train"+str(f) + ".csv", sep="\t", index_col=False)
  df_valid = pd.read_csv(FOLDS_PATH + "test"+str(f) + ".csv", sep="\t", index_col=False)
  print("Fold",f,'\n')
  data_lm = TextLMDataBunch.from_df(path=FOLDER,
                                  train_df=df_train,
                                  valid_df=df_valid, 
                                  label_cols='y', 
                                  text_cols='text')

  lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.2)
  lm.save_encoder(FOLDER+'language_model')

  data_clas = TextClasDataBunch.from_df(path=FOLDER,
                                      train_df=df_train, 
                                      valid_df=df_valid, 
                                      vocab=data_lm.train_ds.vocab,
                                      label_cols='y',
                                      text_cols='text')
  
  learner = text_classifier_learner(data_clas, 
                                 AWD_LSTM,
                                 drop_mult=0.2,                                 
                                 metrics=[accuracy,
                                          Precision(average="macro"), 
                                          Recall(average="macro"), 
                                          FBeta(average="macro", beta=1)])  
  learner.load_encoder(FOLDER+'language_model')
  # print(learner.summary())
  # learner.lr_find()
  # learner.recorder.plot()
  learner.fit_one_cycle(5, max_lr=1e-2)
  

  # loss_value, acc_value, prec_value, rec_value, f1_value = learner.validate()
  # val_acc.append(acc_value.item())
  # val_prec.append(prec_value.item())
  # val_rec.append(rec_value.item())
  # val_f1.append(f1_value.item())

Training on 10 folds
Fold 0 



epoch,train_loss,valid_loss,accuracy,precision,recall,f_beta,time
0,0.724310,0.635547,0.648980,0.666451,0.654211,0.643992,00:01
1,0.701151,0.674106,0.620408,0.624712,0.615541,0.611051,00:01
2,0.694448,0.627825,0.616327,0.634559,0.622114,0.609237,00:01
3,0.670217,0.608255,0.665306,0.676790,0.669358,0.662829,00:01
4,0.652323,0.600766,0.657143,0.658287,0.658181,0.657137,00:01


Fold 1 



epoch,train_loss,valid_loss,accuracy,precision,recall,f_beta,time
0,0.730560,0.642276,0.644898,0.644491,0.644568,0.644519,00:01
1,0.698563,0.616076,0.653061,0.658409,0.655745,0.652227,00:01
2,0.687719,0.623570,0.669388,0.672227,0.671193,0.669189,00:01
3,0.667247,0.609188,0.661224,0.662657,0.662418,0.661202,00:01
4,0.657734,0.604269,0.665306,0.665966,0.666055,0.665301,00:01


Fold 2 



...

epoch,train_loss,valid_loss,accuracy,precision,recall,f_beta,time
0,0.724687,0.615959,0.696721,0.716768,0.691283,0.685545,00:01
1,0.696364,0.615609,0.655738,0.656731,0.653484,0.652916,00:01
2,0.684785,0.642239,0.614754,0.634598,0.620258,0.606181,00:01
3,0.673076,0.625977,0.631148,0.634804,0.633172,0.630527,00:01
4,0.654533,0.611464,0.622951,0.623253,0.623352,0.622925,00:01


Fold 8 



epoch,train_loss,valid_loss,accuracy,precision,recall,f_beta,time
0,0.703716,0.683903,0.504098,0.496273,0.496973,0.475474,00:01
1,0.680159,0.620504,0.655738,0.663899,0.651332,0.647180,00:01
2,0.677522,0.624153,0.655738,0.659488,0.652408,0.650454,00:01
3,0.670295,0.619851,0.639344,0.647336,0.634652,0.629384,00:01
4,0.655497,0.611503,0.647541,0.653824,0.643395,0.639698,00:01


Fold 9 



epoch,train_loss,valid_loss,accuracy,precision,recall,f_beta,time
0,0.753279,0.657695,0.655738,0.656048,0.656174,0.655715,00:01
1,0.708071,0.615497,0.672131,0.681672,0.667743,0.663981,00:01
2,0.689377,0.613592,0.651639,0.651784,0.651937,0.651587,00:01
3,0.670239,0.630304,0.618852,0.618521,0.618577,0.618538,00:01
4,0.661337,0.627349,0.635246,0.634800,0.634719,0.634749,00:01


In [ ]:
lm.summary()

SequentialRNN
Layer (type)         Output Shape         Param #    Trainable 
RNNDropout           [70, 400]            0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
RNNDropout           [70, 1152]           0          False     
______________________________________________________________________
Linear               [70, 2128]           853,328    True      
______________________________________________________________________
RNNDropout           [70, 400]            0          False     
______________________________________________________________________

Total params: 853,328
Total trainable params: 853,328
Total non-trainable params: 0
Optimized with 'torch.optim.adam.Adam', betas=(0.9, 0.99)
Using true weight decay as discussed in https://www.fast.ai/2018/07/02/adam-weight-decay/ 
Loss function

#Salvando resultados em arquivo

In [ ]:
created_at = ctime()
created_at = created_at.replace(' ', '-')
name = "{}-{}-{}-{}-results.csv".format("LSTM", SIZE, C, created_at)
with open(os.path.join(FOLDER, 'results', name), 'w') as f:
  f.write("folds, acc, prec, rec, f1\n")
  f.write("{},{},{},{},{}\n".format(folds, np.mean(val_acc), np.mean(val_prec), np.mean(val_rec), np.mean(val_f1)))
  f.write("{},{},{},{},{}\n".format('-', acc, prec, rec, f1))